# Step5 演習02 MNISTの多クラス分類（TensorFlow）
---

本演習では、手書き数字のMNISTをRNNを用いて分類します。

**はじめに**
- for文やwhile文の利用は明示的な利用指示がない場所での利用は避けてください。

**本演習の目的**
- RNNをTensorFlowの関数を使って実装する

**【課題１】** 採点を実行してください。

In [1]:
#Coursedele-02 Step5 QuestionNumber6 15cd7b9f15ccd5baee95171f0de7e962
print("この課題の採点は全員正解とします。")

この課題の採点は全員正解とします。


** ファイルを保存後 **、次のセルを実行（Shift+Enter）で採点を行います。

In [2]:
%%bash
./validation_client.py dele-02 5 6 Step5_02.ipynb api.internal.zero2one.jp

Congratulations!
We give you 10 points out of 10 points.



## ライブラリのインストール

まずはじめに、本演習で利用するライブラリのインポートを行います。

- [numpy](http://www.numpy.org) 数値計算を行うための基本パッケージの公式ドキュメント
- [matplotlib](http://matplotlib.org) グラフ描画ライブラリの基本パッケージの公式ドキュメント
- [tensorflow](https://www.tensorflow.org/) 機械学習用のライブラリの公式ドキュメント


`%matplotlib inline` はnotebook上で使える[magic function](http://ipython.readthedocs.io/en/stable/interactive/magics.html)の一つで、これによりmatplotlibをインタラクティブに使うことできます。

In [3]:
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## データセット

いままでと同様にMNISTデータセットを読み込みます。

In [4]:
# データの読み込み
mnist = input_data.read_data_sets("MNIST_data/", 
                                  one_hot=True, 
                                  validation_size=0)

# データのシャッフル
permutation = np.random.permutation(mnist.train._images.shape[0])
mnist.train._images = mnist.train.images[permutation]
mnist.train._labels = mnist.train.labels[permutation]

#データの正規化
mean = np.mean(mnist.train.images)
std = np.std(mnist.train.images)
mnist.train._images = (mnist.train.images-mean)/std
mnist.test._images = (mnist.test.images-mean)/std

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## モデルの構築

**【確認１】**　ここでは演習01で実装したRNNをTensorFlowを用いて実装します。

`def RNN`では、
- 入力として`x`を受け取ります
- `x`の形（batchsize, 縦、横）を[tf.reshape](https://www.tensorflow.org/api_docs/python/tf/reshape)を用いて変更します。
- `x`を[tf.unstack](https://www.tensorflow.org/api_docs/python/tf/unstack)を用いて行毎に分割します。
- `rnn_cell`: ここでは`tf.cotrib.rnn`から[BasicRNNCell](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicRNNCell)を呼び設定を行います。ここでは引数として`num_units`と`activation`を与えます。`num_units`は隠れ層のユニット数です（100として下さい）。また`activation`は隠れ層の活性化関数です（ここでは`tf.tanh`）。
- `outputs, states`:　用意した`x`と`rnn_cell`を`tf.nn`の[static_rnn](https://www.tensorflow.org/api_docs/python/tf/nn/static_rnn)に渡します。また、引数として`dtype=tf.float32`も渡して下さい。
- `V`, `c`：　これまでで下の式で言う$\boldsymbol{h}^{(t)}$まで計算しましたが、$\boldsymbol{o}^{(t)}$は計算されていません。なのでここではこの式用に$\boldsymbol{c}$と$\boldsymbol{V}$を用意します。

$$
\begin{eqnarray}
\boldsymbol{a}^{(t)}&=&\boldsymbol{b}+\boldsymbol{h}^{(t-1)}\boldsymbol{W}+\boldsymbol{x}^{(t)}\boldsymbol{U}\\
\boldsymbol{h}^{(t)}&=&tanh(\boldsymbol{a}^{(t)})\\
\boldsymbol{o}^{(t)}&=&\boldsymbol{c}+\boldsymbol{h}^{(t)}\boldsymbol{V}\\
\boldsymbol{\hat{y}}^{(t)}&=&softmax(\boldsymbol{o}^{(t)})
\end{eqnarray}
$$

- `o`: 最後に`output[-1]`と`V`を[tf.matmul](https://www.tensorflow.org/api_docs/python/tf/matmul)でかけて`c`を足します。

In [18]:
def RNN(x):
    ###############START CODE HERE###############
    #適切な 関数・変数・値 などにNoneを書き換えてください。
    # (batch_size, 28*28) -> (batch_size, 28, 28)
    x = tf.reshape(x, (-1, 28, 28))
    
    # 行毎に分割
    x = tf.unstack(x, 28, 1)
    
    # RNNの設定
    rnn_cell = tf.contrib.rnn.BasicRNNCell(num_units=100, activation=tf.tanh)

    # RNNモデルの作成
    outputs, states = tf.nn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # 最終層用の重みとバイアス(V, c)
    std = np.sqrt(6./(100+10))
    V = tf.Variable(tf.random_uniform([100, 10], minval=-std, maxval=std))
    c = tf.Variable(tf.zeros(10))

    o = tf.matmul(outputs[-1], V) + c
    ################END CODE HERE################
    return o

## モデルの準備

### セッションの作成

In [19]:
tf.reset_default_graph()
sess  = tf.InteractiveSession()

### placeholderの用意

In [20]:
x = tf.placeholder("float", [None, 28*28])
y = tf.placeholder("float", [None, 10])

### モデルの読み込み


上記で用意した`def rnn()`を呼びます

In [21]:
preds = RNN(x)

### コスト関数の定義と最適化

CNNの演習と同様にコスト関数にはクロスエントロピー誤差関数を使用します。
- `cost`: ここでは [tf.nn.softmax_cross_entropy_with_logits](https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits) を使用します。また、このままではクラス分の損失があるので [tf.reduce.mean](https://www.tensorflow.org/api_docs/python/tf/reduce_mean) を使って平均を求めます。

次にこの`cost`をADAMを使って最小化します。
- `optimizer`: [tf.train.AdamOptimzer](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer)を追加します。引数には`learning_rate`を渡します。また、これを`.mnimize(cost)`を後尾に書き加えます。

In [22]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

### 精度の計算

ここでは精度を計算します。
- `correct_pred`: [tf.equal](https://www.tensorflow.org/api_docs/python/tf/equal)を使い予測結果の`test_pred`と正解ラベルの`labels`を比較します。この際に、まず両方とも[tf.argmax](https://www.tensorflow.org/api_docs/python/tf/argmax)を使いone-hot-vectorの最大値のインデックスを返させます。ここで、`axis=1`とするのを忘れないようにしてください。
- `accuracy`: `correct_pred`はbool型（[True, False, True...])となっているので[tf.cast](https://www.tensorflow.org/api_docs/python/tf/cast)を使ってtf.float32に変換させます。そして、[tf.reduce_mean](https://www.tensorflow.org/api_docs/python/tf/reduce_mean)を使って精度を求めます。

In [23]:
correct_pred = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

## 学習

**【確認２】**　ここでは今までの演習のコードなどを参考にしながら学習のコードを作成します。

- `mnist.train.next_batch`で次のミニバッチを取り出せます。
- (任意）10ステップごとにログを出すコードを書いて下さい
- 最後にテストデータをモデルに渡し精度を計算して下さい。

In [24]:
%%time
sess.run(tf.global_variables_initializer())
# バッチサイズ
batch_size = 250
# １エポックあたりのステップ数
total_step_train = int(len(mnist.train.labels)/batch_size)
for epoch in range(5):
    for interation in range(total_step_train):
        ###############START CODE HERE###############
        #適切な 関数・変数・値 などにNoneを書き換えてください。
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if interation % 10 == 0:
            acc , _cost = sess.run([accuracy, cost], 
                                feed_dict={x: batch_x, y: batch_y})
            print('\r step: {} / cost: {:.6f} / acc: {:.5f}'.format(interation, _cost, acc), end='')
        ################END CODE HERE################
# テスト
###############START CODE HERE###############
#適切な 関数・変数・値 などにNoneを書き換えてください。
test_acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
################END CODE HERE################
print("\nTest Accuracy: {:.3f}".format(test_acc))

 step: 230 / cost: 0.137016 / acc: 0.96000
Test Accuracy: 0.963
CPU times: user 1min 21s, sys: 6.86 s, total: 1min 28s
Wall time: 46.8 s
